In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image


In [2]:
IMG_SIZE = 224
classes = ["AnnualCrop", "Forest", "HerbaceousVegetation", "Highway", 
           "Industrial", "Pasture", "PermanentCrop", "Residential", "River", "SeaLake"]

model = tf.keras.models.load_model("efficient_model_96.keras")


In [3]:
def predict_class(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_arr = tf.keras.preprocessing.image.img_to_array(img)
    img_arr = img_arr / 255.0
    img_arr = np.expand_dims(img_arr, axis=0)
    pred = model.predict(img_arr)
    return classes[np.argmax(pred)]

In [ ]:
folder = "test_images/"
image_files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]

data = []
for f in image_files:
    true_class = f.split("_")[0]   
    data.append({
        "file": f,
        "true_class": true_class
    })

df = pd.DataFrame(data)


In [10]:
regions = ["Cairo", "Delta", "Sinai", "Upper_Egypt", "Western_Desert", "Red_Sea"]

df["region"] = df["true_class"].apply(lambda x: random.choice(regions))


In [12]:
df["pred_class"] = df["file"].apply(lambda f: predict_class(os.path.join(folder, f)))
df["correct"] = (df["true_class"] == df["pred_class"]).astype(int)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [16]:
region_acc = df.groupby("region")["correct"].mean().reset_index()
region_acc.rename(columns={"correct": "accuracy"}, inplace=True)
region_acc


,region,accuracy
0,Cairo,0.933333
1,Delta,1.000000
2,Red_Sea,1.000000
3,Sinai,1.000000
4,Upper_Egypt,1.000000
5,Western_Desert,1.000000


# classified land types as color-coded images. 

In [18]:
egypt_map = {
    "Cairo": [1, 2],
    "Delta": [2, 2],
    "Sinai": [3, 2],
    "Upper_Egypt": [1, 1],
    "Western_Desert": [2, 1],
    "Red_Sea": [3, 1],
}

region_acc["x"] = region_acc["region"].apply(lambda r: egypt_map[r][0])
region_acc["y"] = region_acc["region"].apply(lambda r: egypt_map[r][1])

fig = px.scatter(region_acc,
                 x="x", y="y",
                 size=[0.4]*len(region_acc),
                 color="accuracy",
                 text="region",
                 color_continuous_scale="Viridis",
                 title="Egypt Regions – Land Classification Accuracy")

fig.update_traces(textposition="top center")
fig.update_layout(yaxis=dict(visible=False),
                  xaxis=dict(visible=False),
                  width=700, height=500)

fig.show()


In [ ]:

region_acc_sorted = region_acc.sort_values(by="accuracy", ascending=False)
fig_bar = px.bar(region_acc_sorted,
                 x="region", y="accuracy",
                 text=region_acc_sorted["accuracy"].apply(lambda x: f"{x:.2f}"),
                 color="accuracy",
                 color_continuous_scale="Viridis",
                 title="Accuracy by Region")
fig_bar.show()


In [20]:
fig3 = px.pie(df, names="pred_class", title="Prediction Distribution")
fig3.show()


# statistics about the accuracy of each class in different regions.

In [ ]:

class_acc_list = []

for region in regions:
    df_region = df[df["region"] == region]
    total_images = len(df_region)
    class_counts = df_region.groupby("true_class")["correct"].agg(["sum", "count"]).reset_index()
    for _, row_class in class_counts.iterrows():
        class_acc_list.append({
            "Region": region,
            "Class": row_class["true_class"],
            "Accuracy": row_class["sum"]/row_class["count"] if row_class["count"] > 0 else np.nan,
            "Images": row_class["count"]
        })

df_class_acc = pd.DataFrame(class_acc_list)

df_class_acc = df_class_acc.pivot(index="Class", columns="Region", values="Accuracy").round(3)
df_class_acc.fillna("-", inplace=True)

print("=== Class-wise Accuracy per Region ===")
print(df_class_acc)


=== Class-wise Accuracy per Region ===
Region                Cairo Delta Red_Sea Sinai Upper_Egypt Western_Desert
Class                                                                     
AnnualCrop              1.0   1.0       -   1.0         1.0              -
Forest                  1.0   1.0     1.0     -         1.0            1.0
HerbaceousVegetation    1.0   1.0     1.0   1.0           -            1.0
Highway                 1.0     -     1.0     -         1.0            1.0
Industrial              1.0   1.0     1.0   1.0         1.0            1.0
Pasture                 1.0     -       -   1.0           -            1.0
PermanentCrop           1.0     -       -   1.0         1.0            1.0
Residential             1.0     -       -   1.0           -            1.0
River                   1.0     -     1.0   1.0           -            1.0
SeaLake                 0.0   1.0     1.0     -         1.0            1.0
